In [1]:
#import packages
import os
import pandas as pd
from sklearn.model_selection import KFold
import cv2 #opencv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from numpy import asarray
import numpy as np

In [2]:
#make empty lists to prepare input and outputs data
path = []
model = []
airplane = []

In [3]:
#prepare input and outputs data (store picture's path and validation outputs)
def each_plane(zmodel, zairplane, zpath):
    corepath = "C:/Users/antoi/Google Drive/Computer Vision/airplane scanner/" + zmodel + "/" + zairplane + "/" + zpath + "/"
    arr = os.listdir(corepath)
    curr_path = []
    for i in range(len(arr)):
        curr_path.append(corepath + arr[i])
    path.extend(curr_path)
    
    curr_model = []
    for i in range(len(arr)):
        curr_model.append(zmodel)
    model.extend(curr_model)
    
    curr_airplane = []
    for i in range(len(arr)):
        curr_airplane.append(zairplane)
    airplane.extend(curr_airplane)

In [4]:
each_plane("civi", "727", "boeing 727 airplane")
each_plane("civi", "707", "boeing 707 airplane")
each_plane("civi", "380", "airbus 380 airplane")
each_plane("civi", "320", "airbus 320 airplane")

each_plane("mili", "su-37", "su-37 airplane")
each_plane("mili", "f-22", "f-22 airplane")
each_plane("mili", "mirage-2000", "mirage 2000 airplane")
each_plane("mili", "rafale", "rafale airplane")

In [5]:
#transform the pictures into a list of array
X = []

for i in range(len(path)):
    img = cv2.imread(path[i])
    img = asarray(img)
    mean = img.mean()
    img = img - mean
    X.append(img)
    
X = np.asarray(X).astype(np.float32)

In [6]:
#transform outputs label intop numbers; make a dictionary of relation between labels and their given number.
from sklearn import preprocessing

print(airplane[1])

le2 = preprocessing.LabelEncoder()
le2.fit(airplane)

y = le2.transform(airplane)
y = np.asarray(y).astype(np.float32)

y_map = dict(zip(le2.classes_, le2.transform(le2.classes_)))
print(y_map)
y_keys, y_values = zip(*y_map.items())
#print(y_keys)
#print(y_values)

print(y[1])

from keras.utils import to_categorical
y = to_categorical(y)

print(y[1])
print(type(y[1]))
print(np.argmax(y[1]))
dd = np.argmax(y[1])
print(dd)
print(y_keys[dd])

727
{'320': 0, '380': 1, '707': 2, '727': 3, 'f-22': 4, 'mirage-2000': 5, 'rafale': 6, 'su-37': 7}
3.0


Using TensorFlow backend.


[0. 0. 0. 1. 0. 0. 0. 0.]
<class 'numpy.ndarray'>
3
3
727


In [7]:
#Split X & y for training and test.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
#prepare inputs and outputs shape values
n_inputs = X.shape[1]
n_outputs = len(list(le2.classes_))
entry = (X.shape[1], X.shape[2], X.shape[3])
exit = len(list(le2.classes_))
print(n_inputs)
print(n_outputs)
print(entry)
print(exit)

256
8
(256, 256, 3)
8


In [10]:
#import model packages
from keras.layers import Dense
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD

In [25]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#define the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    #input & 1st block
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=entry))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    #2nd block
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    #3rd block
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    #preparing exit
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #exit
    model.add(Dense(exit, activation='softmax'))
    #compile
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt , metrics=['accuracy'])
    return model

#del(y_pred)

n_epochs = 20

#train the model
def evaluate_model(X_train, y_train, X_test, y_test):
    results = list()
    n_inputs, n_outputs = X.shape[1], exit
    #call the previously defined model
    model = get_model(n_inputs, n_outputs)
    #train the model
    print('Train...')
    model.fit(X_train, y_train, verbose=1, epochs=n_epochs)
    #test the model, return a confusion matrix
    print('Evaluate...')
    y_pred = model.predict_classes(X_test, verbose=1)
    #y_pred = np.argmax(y_pred, axis=1)
    #y_test = np.asarray(y_test).astype(int)
    y_test = np.argmax(y_test, axis=1)
    uu = accuracy_score(y_test, y_pred)
    print('accuracy score on X_test is :', uu)
    zz = confusion_matrix(y_test, y_pred, labels=y_values)
    zz = pd.DataFrame(zz)
    zz = zz.set_axis(list(le2.classes_), axis='columns', inplace=False)
    zz = zz.set_axis(list(le2.classes_), axis='index', inplace=False)
    print(zz)
    return results

In [26]:
#compile and train the model
results = evaluate_model(X_train, y_train, X_test, y_test)

Train...
Epoch 1/20
3840/3840 [==============================] - 50s 13ms/step - loss: 2.1042 - accuracy: 0.3289
Epoch 2/20
3840/3840 [==============================] - 50s 13ms/step - loss: 1.2532 - accuracy: 0.5617
Epoch 3/20
3840/3840 [==============================] - 50s 13ms/step - loss: 0.8031 - accuracy: 0.7404
Epoch 4/20
3840/3840 [==============================] - 50s 13ms/step - loss: 0.5402 - accuracy: 0.8318
Epoch 5/20
3840/3840 [==============================] - 50s 13ms/step - loss: 0.4168 - accuracy: 0.8706
Epoch 6/20
3840/3840 [==============================] - 49s 13ms/step - loss: 0.3373 - accuracy: 0.8984
Epoch 7/20
3840/3840 [==============================] - 50s 13ms/step - loss: 0.3036 - accuracy: 0.9115
Epoch 8/20
3840/3840 [==============================] - 49s 13ms/step - loss: 0.2875 - accuracy: 0.9109
Epoch 9/20
3840/3840 [==============================] - 50s 13ms/step - loss: 0.2536 - accuracy: 0.9190
Epoch 10/20
3840/3840 [==============================] 

In [ ]:
#[print(i.shape, i.dtype) for i in model.inputs]
#[print(o.shape, o.dtype) for o in model.outputs]
#[print(l.name, l.input_shape, l.dtype) for l in model.layers]